In [2]:
import pandas as pd
fatal_df = pd.read_csv("data/aviationaccidentdata.csv")
nonfatal_df = pd.read_csv("data/nonfatalsdata.csv")
fatal_df.info()
nonfatal_df.info()
fatal_df.head()

cols_to_drop = ["Operator", "Flight number", "Aircraft flight hours", "Aircraft flight cycles", "MSN", "Registration"]
fatal_df.drop(columns=cols_to_drop, inplace=True, errors='ignore')
fatal_df["Date"] = pd.to_datetime(fatal_df["Date & Time"], errors="coerce")
fatal_df["Year"] = fatal_df["Date"].dt.year
fatal_df["Month"] = fatal_df["Date"].dt.month
fatal_df["Total_onboard"] = fatal_df["Crew on board"] + fatal_df["Pax on board"]
fatal_df["Total_fatalities"] = (
    fatal_df["Crew fatalities"] + fatal_df["Pax fatalities"] + fatal_df["Other fatalities"]
)

# Recalculate after this
fatal_df["Fatality_Rate"] = fatal_df["Total_fatalities"] / fatal_df["Total_onboard"]
fatal_df.head()
fatal_df["Date_Cleaned"] = fatal_df["Date & Time"].str.replace("LT", "").str.strip()
fatal_df["Date"] = pd.to_datetime(fatal_df["Date_Cleaned"], errors="coerce")
fatal_df[fatal_df["Fatality_Rate"] > 1]
fatal_df["Location"] = fatal_df["Location"].str.replace("\n", " ").str.strip()
fatal_df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 31 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   URL                            32 non-null     object 
 1   Date & Time                    32 non-null     object 
 2   Type of aircraft               32 non-null     object 
 3   Operator                       2 non-null      object 
 4   Registration                   32 non-null     object 
 5   Flight Phase                   32 non-null     object 
 6   Flight Type                    32 non-null     object 
 7   Survivors                      32 non-null     object 
 8   Site                           32 non-null     object 
 9   Schedule                       32 non-null     object 
 10  MSN                            32 non-null     object 
 11  YOM                            31 non-null     float64
 12  Flight number                  9 non-null      objec

/var/folders/22/rfrfv2d561vc_9q4g24pjrbw0000gn/T/ipykernel_1506/337228089.py:10: FutureWarning: Parsed string "Jun 12, 2025 at 1338 LT" included an un-recognized timezone "LT". Dropping unrecognized timezones is deprecated; in a future version this will raise. Instead pass the string without the timezone, then use .tz_localize to convert to a recognized timezone.
  fatal_df["Date"] = pd.to_datetime(fatal_df["Date & Time"], errors="coerce")


,URL,Date & Time,Type of aircraft,Flight Phase,Flight Type,Survivors,Site,Schedule,YOM,Location,...,Copilot / Total hours on type,Probable cause,Final Report,Date,Year,Month,Total_onboard,Total_fatalities,Fatality_Rate,Date_Cleaned
0,https://www.baaa-acro.com/crash/crash-boeing-7...,"Jun 12, 2025 at 1338 LT",Boeing 787-8 Dreamliner,Takeoff (climb),Scheduled Revenue Flight,Yes,City,Ahmedabad - London,2013.0,Ahmedabad-Sardar Vallabhbhai Patel Gujarat,...,NaN,NaN,NaN,2025-06-12 13:38:00,2025.0,6.0,242,279,1.152893,"Jun 12, 2025 at 1338"
1,https://www.baaa-acro.com/crash/crash-boeing-7...,"Aug 7, 2020 at 1941 LT",Boeing 737-800,Landing (descent or approach),Charter/Taxi (Non Scheduled Revenue Flight),Yes,Airport (less than 10 km from airport),Dubai - Kozhikode,2006.0,Kozhikode-Calicut Kerala,...,1723.0,The probable cause of the accident was the non...,VT-AXH.pdf12.35 MB,2020-08-07 19:41:00,2020.0,8.0,190,21,0.110526,"Aug 7, 2020 at 1941"
2,https://www.baaa-acro.com/crash/crash-boeing-7...,"May 22, 2010 at 0605 LT",Boeing 737-800,Landing (descent or approach),Scheduled Revenue Flight,Yes,Airport (less than 10 km from airport),Dubai - Mangalore,2007.0,Mangalore-Bajpe Karnataka,...,3319.0,The Court of Inquiry determines that the cause...,VT-AXV.pdf4.43 MB,2010-05-22 06:05:00,2010.0,5.0,166,158,0.951807,"May 22, 2010 at 0605"
3,https://www.baaa-acro.com/crash/crash-antonov-...,"Jun 3, 2019 at 1300 LT",Antonov AN-32,Flight,Military,No,Mountains,Jorhat - Mechuka,1987.0,Lipo Arunachal Pradesh,...,NaN,NaN,NaN,2019-06-03 13:00:00,2019.0,6.0,13,13,1.000000,"Jun 3, 2019 at 1300"
4,https://www.baaa-acro.com/crash/crash-beechcra...,"Jun 28, 2018 at 1310 LT",Beechcraft 90 King Air,Landing (descent or approach),Test,No,City,Juhu - Juhu,1995.0,Mumbai Maharashtra,...,156.0,The probable cause of the accident was stall c...,VT-UPZ.pdf57.41 MB,2018-06-28 13:10:00,2018.0,6.0,4,4,1.000000,"Jun 28, 2018 at 1310"


In [7]:
# View columns with missing data
fatal_df.isnull().sum()

# Drop rows where Date or Total_onboard is missing (critical fields)
fatal_df.dropna(subset=["Date", "Total_onboard"], inplace=True)

# Optionally fill others
fatal_df["YOM"] = fatal_df["YOM"].fillna(fatal_df["YOM"].median())


fatal_df.sort_values(by="Date", ascending=True, inplace=True)
fatal_df.reset_index(drop=True, inplace=True)
fatal_df.info()
fatal_df.head()
fatal_df.to_csv("data/cleaned_fatal_crashes.csv", index=False)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 32 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   URL                            29 non-null     object        
 1   Date & Time                    29 non-null     object        
 2   Type of aircraft               29 non-null     object        
 3   Flight Phase                   29 non-null     object        
 4   Flight Type                    29 non-null     object        
 5   Survivors                      29 non-null     object        
 6   Site                           29 non-null     object        
 7   Schedule                       29 non-null     object        
 8   YOM                            29 non-null     float64       
 9   Location                       29 non-null     object        
 10  Country                        29 non-null     object        
 11  Region               

In [10]:
cols_to_fix = [
    "Captain / Total flying hours", "Copilot / Total flying hours",
    "Captain / Total hours on type", "Copilot / Total hours on type"
]

for col in cols_to_fix:
    fatal_df[col] = pd.to_numeric(fatal_df[col], errors="coerce")
    
fatal_df["Year"] = fatal_df["Year"].astype("Int64")
fatal_df["Month"] = fatal_df["Month"].astype("Int64")

fatal_df.drop(columns=["Date_Cleaned", "URL", "Final Report", "Site", "Schedule"], inplace=True, errors="ignore")

fatal_df.head()
fatal_df.to_csv("data/updated_fatal_crashes.csv", index=False)


In [15]:
cols = [
    "Captain / Total flying hours", "Copilot / Total flying hours",
    "Captain / Total hours on type", "Copilot / Total hours on type"
]

for col in cols:
    fatal_df[col] = pd.to_numeric(fatal_df[col], errors='coerce')
fatal_df.drop(columns=["Date & Time", "Survivors"], inplace=True, errors="ignore")
fatal_df["Date"] = pd.to_datetime(fatal_df["Date"])
fatal_df.sort_values("Date", inplace=True)
fatal_df.reset_index(drop=True, inplace=True)
fatal_df.to_csv("data/final_fatal_crashes.csv", index=False)

In [17]:
# Create a preview column with first 300 characters
fatal_df["Circumstances_Short"] = fatal_df["Circumstances"].str.slice(0, 300) + "..."
fatal_df.to_csv("data/final_fatal_crashes.csv", index=False)

In [21]:
fatal_df = fatal_df[fatal_df["Total_onboard"] > 0]
fatal_df["Fatality_Rate"] = fatal_df["Total_fatalities"] / fatal_df["Total_onboard"]
fatal_df["Onboard_fatalities"] = fatal_df["Crew fatalities"] + fatal_df["Pax fatalities"]
fatal_df["Fatality_Rate_Onboard"] = fatal_df["Onboard_fatalities"] / fatal_df["Total_onboard"]
fatal_df.to_csv("data/fatal_crashes.csv", index=False)